### Importing necessary libararies

In [1]:
import json
import glob
import numpy as np
import cv2
import copy
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

### Finding Frames

In [6]:
# finding the number of frames for each video in dataset.
video_files = glob.glob('/Users/karthiksagar/DEEP-FAKE-DETECTION/dataset-real/*.mp4')
video_files1 = glob.glob('/Users/karthiksagar/DEEP-FAKE-DETECTION/dataset-fake/*.mp4')
video_files += video_files1

frame_count = []
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    if (int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150):
        video_files.remove(video_file)
        continue
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

print("Frames : ", frame_count)
print("Total Number of Videos : ", len(frame_count))
print("Average frames per video : ", np.mean(frame_count))

Frames :  [464, 520, 420, 456, 401, 461, 314, 350, 529, 303, 469, 326, 459, 479, 534, 520, 326, 459, 534, 350, 529, 303, 469, 303, 469, 350, 529, 479, 534, 459]
Total Number of Videos :  30
Average frames per video :  436.6


### Extracting Frames

In [9]:
def frame_extract(path):
    video_object = cv2.VideoCapture(path)
    success = 1
    while success:
        success, image = video_object.read()
        if success:
            yield image

#!mkdir '/Users/karthiksagar/DEEP-FAKE-DETECTION/Face-Only-Data'

## process the frames to create face only videos

def create_face_only_videos(path_list, output_dir):
    present_count = glob.glob(output_dir+'*.mp4')
    print("No. of Videos already present : ", len(present_count))
    for path in tqdm(path_list):
        output_path = os.path.join(output_dir, path.split('/')[-1])
        file_exists = glob.glob(output_path)
        if (len(file_exists) != 0):
            print("File Alreday Exists at : ", output_path)
            continue

        frames = []
        flag = 0
        face_all = []
        frames1 = []
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc('M', 'J','P','G'), 30, (112,112))
        for idx, frame in enumerate(frame_extract(path)):
            if(idx <= 150):
                frames.append(frame)
                if (len(frames) == 4):
                    faces = face_recognition.batch_face_locations(frames)
                    for i, face in enumerate(faces):
                        if (len(face) == 0):
                            top, right, bottom, left = face[0]
                        try:
                            out.write(cv2.resize(frames[i][top: bottom, left: right, :], (112,112)))
                        except:
                            pass
                    frames = []
        try:
            del top, right, bottom, left
        except:
            pass
        out.release()

### Extracting Face only videos

In [5]:
create_face_only_videos(video_files, "/Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/")

No. of Videos already present :  93


  0%|          | 0/299 [00:00<?, ?it/s]

File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id13_0007.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id4_0001.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id13_0013.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id12_0006.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id13_0012.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id13_0006.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id4_0000.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id1_0008.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id0_0008.mp4
File Alreday Exists at :  /Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data/id13_001

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FF

In [10]:
create_face_only_videos(glob.glob('/Users/karthiksagar/DEEP-FAKE-DETECTION/dataset-real/*.mp4'), "/Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data-Real/")

No. of Videos already present :  0


  0%|          | 0/15 [00:00<?, ?it/s]

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


KeyboardInterrupt: 

In [ ]:
create_face_only_videos(glob.glob('/Users/karthiksagar/DEEP-FAKE-DETECTION/dataset-fake/*.mp4'), "/Users/karthiksagar/DEEP-FAKE-DETECTION/Celeb-Face-Only-Data-Fake/")